<a href="https://colab.research.google.com/github/cartmarsh/MachineLearning1/blob/main/HumanProteinAtlasDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

In [3]:
!kaggle competitions download -c human-protein-atlas-image-classification

100% 17.5G/17.5G [02:35<00:00, 171MB/s]
100% 17.5G/17.5G [02:35<00:00, 121MB/s]


In [4]:
!ls

human-protein-atlas-image-classification.zip  kaggle.json  sample_data


# Outline

1. Explore data
2. transform data
2. create DataSet
3. create DataLoader
4. create Model
5. make Submisson file
6. submit

## Exploring the data

In [ ]:
!unzip human-protein-atlas-image-classification.zip 

In [6]:
!ls

human-protein-atlas-image-classification.zip  kaggle.json  sample_data


In [9]:
import pandas as pd

train_df = pd.read_csv("train.csv", index_col="Id")
train_df.head(10)

Target
Id                                           
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18
001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0        0
0020af02-bbba-11e8-b2ba-ac1f6b6435d0     25 2
002679c2-bbb6-11e8-b2ba-ac1f6b6435d0        0
00285ce4-bba0-11e8-b2b9-ac1f6b6435d0      2 0
002daad6-bbc9-11e8-b2bc-ac1f6b6435d0        7

In [10]:
train_df["Target"] = train_df["Target"].str.split(" ").map(lambda x: list(map(int, x)))

In [11]:
train_df.head(10)

Target
Id                                                
00070df0-bbc3-11e8-b2bc-ac1f6b6435d0       [16, 0]
000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  [7, 1, 2, 0]
000a9596-bbc4-11e8-b2bc-ac1f6b6435d0           [5]
000c99ba-bba4-11e8-b2b9-ac1f6b6435d0           [1]
001838f8-bbca-11e8-b2bc-ac1f6b6435d0          [18]
001bcdd2-bbb2-11e8-b2ba-ac1f6b6435d0           [0]
0020af02-bbba-11e8-b2ba-ac1f6b6435d0       [25, 2]
002679c2-bbb6-11e8-b2ba-ac1f6b6435d0           [0]
00285ce4-bba0-11e8-b2b9-ac1f6b6435d0        [2, 0]
002daad6-bbc9-11e8-b2bc-ac1f6b6435d0           [7]

In [12]:
label_names = ["Nucleoplasm", "Nuclear membrane", "Nucleoli", "Nucleoli fibrillar center", 
               "Nuclear speckles", "Nuclear bodies", "Endoplasmic reticulum", 
               "Golgi apparatus", "Peroxisomes", "Endosomes","Lysosomes", 
               "Intermediate filaments", "Actin filaments", "Focal adhesion sites", 
               "Microtubules", "Microtubule ends", "Cytokinetic bridge", "Mitotic spindle", 
               "Microtubule organizing center", "Centrosome", "Lipid droplets", 
               "Plasma membrane", "Cell junctions", "Mitochondria", "Aggresome",   
               "Cytosol", "Cytoplasmic bodies", "Rods & rings"]

Creating frequencies

In [13]:
import numpy as np

def get_label_freqs(targets, label_names, ascending=None):
    n_classes = len(label_names)
    freqs = np.array([0] * n_classes)
    for lst in targets:
        for c in range(n_classes):
            freqs[c] += c in lst
    data = {
        'name': label_names, 
        'frequency': freqs, 
        'percent': (10000 * freqs / len(targets)).astype(int) / 100.,
    }
    cols = ['name', 'frequency', 'percent']
    df = pd.DataFrame(data, columns=cols)
    if ascending is not None:
        df = df.sort_values(by='frequency', ascending=ascending)
    return df

In [14]:
# Create a frequency table
train_freqs = get_label_freqs(train_df.Target, label_names, ascending=False)
train_freqs

name  frequency  percent
0                     Nucleoplasm      12885    41.46
25                        Cytosol       8228    26.48
21                Plasma membrane       3777    12.15
2                        Nucleoli       3621    11.65
23                   Mitochondria       2965     9.54
7                 Golgi apparatus       2822     9.08
5                  Nuclear bodies       2513     8.08
4                Nuclear speckles       1858     5.97
3       Nucleoli fibrillar center       1561     5.02
19                     Centrosome       1482     4.76
1                Nuclear membrane       1254     4.03
11         Intermediate filaments       1093     3.51
14                   Microtubules       1066     3.43
6           Endoplasmic reticulum       1008     3.24
18  Microtubule organizing center        902     2.90
22                 Cell junctions        802     2.58
12                Actin filaments        688     2.21
13           Focal adhesion sites        537     1.72
16             Cytokinetic bridge        530     1.70
26             Cytoplasmic bodies        328     1.05
24                      Aggresome        322     1.03
17                Mitotic spindle        210     0.67
20                 Lipid droplets        172     0.55
8                     Peroxisomes         53     0.17
9                       Endosomes         45     0.14
10                      Lysosomes         28     0.09
15               Microtubule ends         21     0.06
27                   Rods & rings         11     0.03

In [15]:
TRAIN_DIR = "/train"

In [16]:
# Better approach: use pathlib
from pathlib import Path

DATA_DIR = Path('./')
TRAIN_DIR = DATA_DIR/'train'
TEST_DIR = DATA_DIR/'test'

In [17]:
CHANNELS = ['green', 'red', 'blue', 'yellow']

# Load images for multiple channels
def load_image(image_id, channels=CHANNELS, img_dir=TRAIN_DIR):
    image = np.zeros(shape=(len(channels),512,512))
    for i, ch in enumerate(channels):
        image[i,:,:] = imread(str(img_dir/f'{image_id}_{ch}.png'))
    return image

In [18]:
# Plot multiple images in a grid
def show_image_filters(image, title, figsize=(16,5)):
    fig, subax = plt.subplots(1, 4, figsize=figsize)
    # Green channel
    subax[0].imshow(image[0], cmap="Greens")
    subax[0].set_title(title)
    # Red channel
    subax[1].imshow(image[1], cmap="Reds")
    subax[1].set_title("Microtubules")
    # Blue channel
    subax[2].imshow(image[2], cmap="Blues")
    subax[2].set_title("Nucleus")
    # Orange channel
    subax[3].imshow(image[3], cmap="Oranges")
    subax[3].set_title("Endoplasmatic reticulum")
    return subax

In [19]:
# Use the traning data to show appropriate labels
def get_labels(image_id):
    labels = [label_names[x] for x in train_df.loc[image_id]['Target']]
    return ', '.join(labels)

In [20]:
from imageio import imread
import matplotlib.pyplot as plt

# Look at a sample grid
img_id = 'ac39847a-bbb1-11e8-b2ba-ac1f6b6435d0'
img, title = load_image(img_id), get_labels(img_id)
show_image_filters(img, title);
print(img.shape)

In [22]:
!pwd

/content


In [23]:
import os


def get_unique_ids2(path):
  # Pfad des Ordners, durch den du iterieren möchtest
  folder_path = path

  # Leere Liste, um die eindeutigen IDs zu speichern
  unique_ids = []

  # Iteriere über die Dateinamen im Ordner
  for filename in os.listdir(folder_path):
    # Extraktion der ID aus dem Dateinamen
    file_id = filename.split('/')[-1].split('_')[0]
    
    # Hinzufügen der ID zur Liste, falls sie noch nicht enthalten ist
    if file_id not in unique_ids:
      unique_ids.append(file_id)

  # Ausgabe der Liste mit den eindeutigen IDs
  
  df = pd.DataFrame({'ids': unique_ids})
  print(df)
  return df

In [24]:
df1 = get_unique_ids2("/content/test")




                                        ids
0      585d7eb2-bac9-11e8-b2b8-ac1f6b6435d0
1      998af66c-bac9-11e8-b2b8-ac1f6b6435d0
2      bcf540f4-bac8-11e8-b2b8-ac1f6b6435d0
3      a3c83004-bace-11e8-b2b8-ac1f6b6435d0
4      75d3c12a-bad0-11e8-b2b8-ac1f6b6435d0
...                                     ...
11697  bdc10346-bac9-11e8-b2b8-ac1f6b6435d0
11698  4e90ea86-bace-11e8-b2b8-ac1f6b6435d0
11699  d8c9e3ac-bad2-11e8-b2b8-ac1f6b6435d0
11700  476415aa-bad6-11e8-b2b9-ac1f6b6435d0
11701  70b4b916-bad9-11e8-b2b9-ac1f6b6435d0

[11702 rows x 1 columns]


In [28]:
type(df1)

pandas.core.frame.DataFrame

In [29]:

# Ausgabe des DataFrames
print(df1)

                                        ids
0      585d7eb2-bac9-11e8-b2b8-ac1f6b6435d0
1      998af66c-bac9-11e8-b2b8-ac1f6b6435d0
2      bcf540f4-bac8-11e8-b2b8-ac1f6b6435d0
3      a3c83004-bace-11e8-b2b8-ac1f6b6435d0
4      75d3c12a-bad0-11e8-b2b8-ac1f6b6435d0
...                                     ...
11697  bdc10346-bac9-11e8-b2b8-ac1f6b6435d0
11698  4e90ea86-bace-11e8-b2b8-ac1f6b6435d0
11699  d8c9e3ac-bad2-11e8-b2b8-ac1f6b6435d0
11700  476415aa-bad6-11e8-b2b9-ac1f6b6435d0
11701  70b4b916-bad9-11e8-b2b9-ac1f6b6435d0

[11702 rows x 1 columns]


In [26]:
len(list(set([str(fn).split('/')[-1].split('_')[0] for fn in TEST_DIR.iterdir()])))

11702

In [27]:
!ls


human-protein-atlas-image-classification.zip  sample_submission.csv  train.csv
kaggle.json				      test
sample_data				      train


Basically we have the images and the train.csv and we need to figure out a way to use to create a model.
I want to use pytorch models for this.
- create DataSet
- create DataLoader

https://www.kaggle.com/code/hamzafarook/hpa-new

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
import os
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
from keras import applications
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam

In [2]:
!ls
!cd train
!ls

human-protein-atlas-image-classification.zip  sample_submission.csv  train.csv
kaggle.json				      test
sample_data				      train
human-protein-atlas-image-classification.zip  sample_submission.csv  train.csv
kaggle.json				      test
sample_data				      train


In [3]:
dataframe = pd.read_csv('./train.csv')
dataframe.head(5)

Id   Target
0  00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
1  000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
2  000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
3  000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
4  001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18

In [4]:
INPUT_SHAPE = (512, 512, 3)
BATCH_SIZE = 16
path_to_train = './train/'

In [5]:
dataframe["complete_path"] = path_to_train + dataframe["Id"]
dataframe.head(5)

Id   Target  \
0  00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0   
1  000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0   
2  000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5   
3  000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1   
4  001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18   

                                  complete_path  
0  ./train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0  
1  ./train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  
2  ./train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0  
3  ./train/000c99ba-bba4-11e8-b2b9-ac1f6b6435d0  
4  ./train/001838f8-bbca-11e8-b2bc-ac1f6b6435d0

In [6]:
train, val = train_test_split(dataframe, test_size=0.2, random_state=42)

In [7]:
print(f'Shape of train: {train.shape}')
print(f'Shape of val: {val.shape}')


Shape of train: (24857, 3)
Shape of val: (6215, 3)


In [8]:
def get_clean_data(df):
    targets = []
    paths = []
    for _, row in df.iterrows():
        target_np = np.zeros((28))
        t = [int(t) for t in row.Target.split()]
        target_np[t] = 1
        targets.append(target_np)
        paths.append(row.complete_path)
    return np.array(paths), np.array(targets)

In [9]:
train_path, train_target = get_clean_data(train)
val_path, val_target = get_clean_data(val)

In [10]:
test_data = get_clean_data()

TypeError: ignored

In [ ]:
print(f'Train path shape: {train_path.shape}')
print(f'Train target shape: {train_target.shape}')
print(f'Val path shape: {val_path.shape}')
print(f'Val target shape: {val_target.shape}')

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_path, train_target))
val_data = tf.data.Dataset.from_tensor_slices((val_path, val_target))

In [ ]:
test_data = tf.data.Dataset.from_tensor_slices((test_path))

In [ ]:
def load_data(path, target):
    red = tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_red.png'), channels=1), [2])
    blue = tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_blue.png'), channels=1), [2])
    green = tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_green.png'), channels=1), [2])
    #yellow=tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_yellow.png'), channels=1), [2])
    img = tf.stack((
                red,
                green,
                blue), axis=2)
    return img, target

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.map(load_data, num_parallel_calls=AUTOTUNE)
val_data = val_data.map(load_data, num_parallel_calls=AUTOTUNE)

In [ ]:
test_data = test_data.map

In [ ]:
def image_augment(img, target):
    img = tf.image.random_contrast(img, lower=0.3, upper=2.0)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, max_delta=0.1)
    return img, target
    
train_data = train_data.map(image_augment, num_parallel_calls=AUTOTUNE)

In [ ]:
train_data_batches = train_data.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
val_data_batches = val_data.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
test_data_batches = 

In [ ]:
inception_model = applications.InceptionResNetV2(include_top=False, weights='imagenet')

inception_model.trainable = False

input_layer = Input(shape=INPUT_SHAPE)
x = inception_model(input_layer)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(28, activation='sigmoid')(x)
model = Model(input_layer, output)

model.summary()

In [ ]:
model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
history = model.fit(train_data_batches, validation_data = val_data_batches,steps_per_epoch = 10, epochs=5)

- go through test.csv
- get the id
- get the image from test folder with images
- create a prediction
- write number into submission.csv file